In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [3]:
# Load the saved model
model = load_model('../model/LSTM_model.h5')
print("\nModel loaded!")


Model loaded!


In [5]:
# Define the tokenizer used during training (ensure you use the same tokenizer!)
tokenizer = Tokenizer(num_words=4000)  # Use the same max_features (or num_words) as before

In [17]:
# Example tweet
tweet = "faggot you nigga fatass fucking hate you will kill you fatass cunt"

# Convert the tweet to a sequence using the tokenizer
tweet_seq = tokenizer.texts_to_sequences([tweet])

# Pad the sequence to match the input length of the model
max_len = 30  # Ensure this is the same max_len as used during training
tweet_padded = pad_sequences(tweet_seq, maxlen=max_len)

# Make the prediction
pred_proba = model.predict(tweet_padded)

# Convert the predicted probabilities to class labels
pred_label = np.argmax(pred_proba, axis=1)

# Map the predicted label back to the class name
label_names = ['Negative', 'Neutral', 'Positive']  # Adjust this to match your actual label classes
predicted_class = label_names[pred_label[0]]

# Print the result
print(f"Predicted class: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted class: Neutral


In [24]:
import nltk
import re
import joblib
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Define the preprocess function
def preprocess(text_string):
    """
    Preprocess the tweet text:
    - Remove URLs
    - Remove mentions
    - Replace multiple spaces with one
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    return parsed_text

# Define the basic_tokenize function
def basic_tokenize(tweet):
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    return tweet.split()

# Define the 'tokenize' function used during model training
def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    tokens = [basic_tokenize(t) for t in tweet.split()]
    return tokens

# Define the PosTfidfVectorizer class
class PosTfidfVectorizer(BaseEstimator, TransformerMixin):
    """Get POS tags for tweets and transform via tfidf"""
    
    def __init__(self):
        self._pos_vectorizer = TfidfVectorizer(
            tokenizer=None,
            lowercase=False,
            preprocessor=None,
            ngram_range=(1, 3),
            stop_words=None,
            use_idf=False,
            smooth_idf=False,
            norm=None,
            decode_error='replace',
            max_features=5000,
            min_df=5,
            max_df=0.75,
        )    
    
    def _preprocess(self, X):
        tweet_tags = []
        for t in X:
            tokens = basic_tokenize(preprocess(t))
            tags = nltk.pos_tag(tokens)
            tag_list = [x[1] for x in tags]
            tag_str = " ".join(tag_list)
            tweet_tags.append(tag_str)
        return tweet_tags
    
    def fit(self, X, y=None):
        tweet_tags = self._preprocess(X)
        self._pos_vectorizer.fit(X)
        return self
    
    def transform(self, X, y=None):
        tweet_tags = self._preprocess(X)
        return self._pos_vectorizer.transform(X)

# Load the pre-trained model
model = joblib.load('../model/multiclass_model.pkl')

# Test the model with a sample tweet
tweet = ["This is a sample tweet!"]  # The tweet to classify

# Predict the class of the tweet using the loaded model
prediction = model.predict(tweet)

# Output the prediction
print(f"Predicted class: {prediction[0]}")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xelor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\xelor\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


AttributeError: Can't get attribute 'SentimentVectorizer' on <module '__main__'>